In [1]:
import requests
import pandas as pd
from decimal import Decimal

In [2]:
API_KEY = """Put your API key here"""
url = f'https://min-api.cryptocompare.com/data/ob/l1/top?&api_key={API_KEY}&fsyms=BTC,ETH&tsyms=USD,GBP&e=Coinbase'
requests.get(url).json()

{'Response': 'Success',
 'Message': '',
 'HasWarning': False,
 'Type': 100,
 'RateLimit': {},
 'Data': {'RAW': {'BTC': {'USD': {'BID': '7687.78', 'ASK': '7689.43'},
    'GBP': {'BID': '6050.93', 'ASK': '6060.63'}},
   'ETH': {'USD': {'BID': '242.74', 'ASK': '242.84'},
    'GBP': {'BID': '190.75', 'ASK': '191.43'}}},
  'DISPLAY': {'BTC': {'USD': {'BID': '$ 7,687.78', 'ASK': '$ 7,689.43'},
    'GBP': {'BID': '£ 6,050.93', 'ASK': '£ 6,060.63'}},
   'ETH': {'USD': {'BID': '$ 242.74', 'ASK': '$ 242.84'},
    'GBP': {'BID': '£ 190.75', 'ASK': '£ 191.43'}}}}}

In [3]:
def top_level(fsyms, tsyms, exchange):
    url = f'https://min-api.cryptocompare.com/data/ob/l1/top?&api_key={API_KEY}'
    fsyms_combined = ','.join(fsyms)
    tsyms_combined = ','.join(tsyms)
    url += f'&fsyms={fsyms_combined}&tsyms={tsyms_combined}&e={exchange}'
    return requests.get(url).json()['Data']['RAW']

In [4]:
top_level(fsyms = ['BTC', 'ETH'], tsyms = ['USD', 'GBP'], exchange = 'Coinbase')

{'BTC': {'USD': {'BID': '7687.78', 'ASK': '7689.43'},
  'GBP': {'BID': '6050.93', 'ASK': '6060.63'}},
 'ETH': {'USD': {'BID': '242.74', 'ASK': '242.84'},
  'GBP': {'BID': '190.75', 'ASK': '191.43'}}}

In [5]:
level_one = top_level(fsyms = ['BTC', 'ETH'], tsyms = ['USD', 'GBP'], exchange = 'Coinbase')
float(level_one['BTC']['USD']['ASK']) - float(level_one['BTC']['USD']['BID'])

1.6500000000005457

In [6]:
level_one = top_level(fsyms = ['BTC', 'ETH'], tsyms = ['USD', 'GBP'], exchange = 'Coinbase')
Decimal(level_one['BTC']['USD']['ASK']) - Decimal(level_one['BTC']['USD']['BID'])

Decimal('1.65')

In [7]:
float(Decimal(level_one['BTC']['USD']['ASK']) - Decimal(level_one['BTC']['USD']['BID']))

1.65

In [8]:
def spread(fsym, tsym, exchange):
    url = f'https://min-api.cryptocompare.com/data/ob/l1/top?&api_key={API_KEY}&fsyms={fsym}&tsyms={tsym}&e={exchange}'
    response = requests.get(url).json()
    raw_data = response['Data']['RAW']
    bid = raw_data[fsym][tsym]['BID']
    ask = raw_data[fsym][tsym]['ASK']
    return float(Decimal(ask) - Decimal(bid))

In [9]:
spread('BTC', 'USD', 'Coinbase')

1.65

In [10]:
def multispreads(fsyms, tsyms, exchange):
    level_one = top_level(fsyms, tsyms, exchange)
    spreads = {}
    for fsym in fsyms:
        fsym_dict = {}
        for tsym in tsyms:
            fsym_dict[tsym] = float(Decimal(level_one[fsym][tsym]['ASK']) - Decimal(level_one[fsym][tsym]['BID']))
        spreads[fsym] = fsym_dict
    return spreads

In [11]:
fsyms = ['BTC', 'ETH', 'LTC', 'ETC', 'BCH']
tsyms = ['USD', 'GBP', 'EUR']
exchange = 'Coinbase'
top_level(fsyms, tsyms, exchange)
spreads = multispreads(fsyms, tsyms, exchange)
spreads

{'BTC': {'USD': 1.65, 'GBP': 9.47, 'EUR': 5.66},
 'ETH': {'USD': 0.1, 'GBP': 0.61, 'EUR': 0.15},
 'LTC': {'USD': 0.07, 'GBP': 0.28, 'EUR': 0.01},
 'ETC': {'USD': 0.02, 'GBP': 0.063, 'EUR': 0.021},
 'BCH': {'USD': 0.44, 'GBP': 1.39, 'EUR': 0.6}}

In [12]:
fsyms = ['BTC', 'ETH', 'LTC', 'ETC', 'BCH']
tsyms = ['USD', 'GBP', 'EUR']
exchange = 'Coinbase'
spreads = multispreads(fsyms, tsyms, exchange)
df = pd.DataFrame(spreads)
df

,BTC,ETH,LTC,ETC,BCH
EUR,5.68,0.18,0.03,0.008,0.63
GBP,9.70,0.68,0.31,0.064,1.42
USD,1.65,0.10,0.05,0.020,0.24
